In [ ]:
from google.colab import drive
import pandas as pd
import json
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import sqlite3
import uuid
os.makedirs("/content/drive/MyDrive/grayball", exist_ok=True)

In [ ]:
from zipfile import ZipFile

with ZipFile("/content/drive/MyDrive/grayball/all_json.zip", 'r') as zObject:
    zObject.extractall(path="/content/drive/MyDrive/grayball/all_json/")

In [ ]:
def insert_match_info(cric_data, match_id):

    data = cric_data["info"]
    team1_name = 'team1'
    team2_name = 'team2'

    team1 = data['teams'][0]
    team2 = data['teams'][1]

    data['players'][team1_name] = data['players'].pop(team1)
    data['players'][team2_name] = data['players'].pop(team2)

    conn = sqlite3.connect('/content/drive/MyDrive/grayball/cricket.db')
    cursor = conn.cursor()

    # Extract data into separate variables
    balls_per_over = data.get('balls_per_over', None)
    city = data.get('city', None)
    dates = data['dates'][0] if 'dates' in data and data['dates'] else None
    event_name = data.get('event', {}).get('name', None)
    event_match_number = data.get('event', {}).get('match_number', None)
    gender = data.get('gender', None)
    match_type = data.get('match_type', None)
    match_type_number = data.get('match_type_number', None)
    official_match_referees = ', '.join(data.get('officials', {}).get('match_referees', []))
    official_reserve_umpires = ', '.join(data.get('officials', {}).get('reserve_umpires', []))
    official_tv_umpires = ', '.join(data.get('officials', {}).get('tv_umpires', []))
    official_umpires = ', '.join(data.get('officials', {}).get('umpires', []))

    outcome = data.get('outcome', {})
    outcome_winner = outcome.get('winner', None)
    outcome_wickets = outcome.get('by', {}).get('wickets', None)
    outcome_runs = outcome.get('by', {}).get('runs', None)

    overs = data.get('overs', None)
    player_of_match = ', '.join(data.get('player_of_match', []))
    players_team1 = ', '.join(data['players'].get(team1_name, []))
    players_team2 = ', '.join(data['players'].get(team2_name, []))
    season = data.get('season', None)
    team_type = data.get('team_type', None)
    teams = ', '.join(data.get('teams', []))
    toss_decision = data.get('toss', {}).get('decision', None)
    toss_winner = data.get('toss', {}).get('winner', None)
    venue = data.get('venue', None)

    # Create the table if it doesn't exist
    cursor.execute('''
          CREATE TABLE IF NOT EXISTS match_info (
              match_id TEXT PRIMARY KEY,
              balls_per_over INTEGER,
              city TEXT,
              dates DATE,
              event_name TEXT,
              event_match_number INTEGER,
              gender TEXT,
              match_type TEXT,
              match_type_number INTEGER,
              official_match_referees TEXT,
              official_reserve_umpires TEXT,
              official_tv_umpires TEXT,
              official_umpires TEXT,
              outcome_winner TEXT,
              outcome_wickets INTEGER,
              outcome_runs INTEGER,
              overs INTEGER,
              player_of_match TEXT,
              players_team1 TEXT,
              players_team2 TEXT,
              season TEXT,
              team_type TEXT,
              teams TEXT,
              toss_decision TEXT,
              toss_winner TEXT,
              venue TEXT,
              team1 TEXT,
              team2 TEXT
            )
    ''')

    # Insert data into the table using placeholders
    cursor.execute('''
        INSERT INTO match_info
        (match_id, balls_per_over, city, dates, event_name, event_match_number, gender, match_type, match_type_number,
        official_match_referees, official_reserve_umpires, official_tv_umpires, official_umpires,
        outcome_winner, outcome_wickets, outcome_runs, overs, player_of_match, players_team1, players_team2,
        season, team_type, teams, toss_decision, toss_winner, venue, team1, team2)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''',
    (match_id, balls_per_over, city, dates, event_name, event_match_number, gender, match_type, match_type_number,
    official_match_referees, official_reserve_umpires, official_tv_umpires, official_umpires,
    outcome_winner, outcome_wickets, outcome_runs, overs, player_of_match,
    players_team1, players_team2, season, team_type, teams,
    toss_decision, toss_winner, venue, team1, team2))

    conn.commit()
    conn.close()

In [ ]:
def insert_ball_by_ball(cric_data, match_id):
    ball_by_ball_data = cric_data.get("innings", [])

    # Connect to an SQLite database
    conn = sqlite3.connect('/content/drive/MyDrive/grayball/cricket.db')
    cursor = conn.cursor()

    # Create a table for ball-by-ball data with 'team' and 'powerplay' columns
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS ball_by_ball_data (
            match_id TEXT,
            team TEXT,
            innings_number INTEGER,
            over_number INTEGER,
            powerplay_type TEXT,  -- Type of powerplay (e.g., 'mandatory', 'optional')
            powerplay_number TEXT,  -- Powerplay number or 'no_powerplay'
            ball_number INTEGER,
            batter TEXT,
            batter_number INTEGER,
            non_striker_number INTEGER,
            bowler TEXT,
            bowler_number INTEGER,
            non_striker TEXT,
            runs_batter INTEGER,
            extras_total INTEGER,
            target_overs INTEGER,
            target_runs INTEGER,
            wickets_player_out TEXT,
            wickets_kind TEXT,
            fielder1_wicket TEXT,
            fielder2_wicket TEXT,
            fielder3_wicket TEXT,
            legbyes_extras INTEGER,
            wides_extras INTEGER,
            byes_extras INTEGER,
            noballs_extras INTEGER,
            penalty_extras INTEGER
        )
    ''')
    count_inn = 0
    for innings_data in ball_by_ball_data:
        batsman_order = {}
        batting_order = 1
        bowler_order = {}
        bowling_order = 1
        count_inn += 1

        team = innings_data.get('team', None)
        powerplays = innings_data.get('powerplays', [])

        for over_data in innings_data.get('overs', []):
            over_number = over_data.get('over', None)
            pp = "no_powerplay"
            pp_type = "NA"
            target_info = innings_data.get('target', {})
            target_overs = target_info.get('overs', None)
            target_runs = target_info.get('runs', None)
            pp_dict = sorted(powerplays, key=lambda pp: pp.get('from', 0))
            for num, pp_data in enumerate(pp_dict, start=1):
                if pp_data.get('from', 0) <= over_number < pp_data.get('to', 0):
                    pp = str(num)
                    pp_type = pp_data.get('type', "NA")

            ball_number = 1
            for delivery in over_data.get('deliveries', []):
                batter_name = delivery.get('batter', None)
                bowler_name = delivery.get('bowler', None)
                non_striker_name = delivery.get('non_striker', None)

                # Check if the batsman has been assigned a batting number
                if batter_name not in batsman_order:
                    batsman_order[batter_name] = batting_order
                    batting_order += 1

                # Check if the non-striker has been assigned a batting number
                if non_striker_name not in batsman_order:
                    batsman_order[non_striker_name] = batting_order
                    batting_order += 1

                # Check if the bowler has been assigned a bowling number
                if bowler_name not in bowler_order:
                    bowler_order[bowler_name] = bowling_order
                    bowling_order += 1

                # Check if the

                runs_batter = delivery.get('runs', 0)
                wickets = delivery.get('wickets', [])
                extras = delivery.get('extras', {})
                runs_extras = sum(extras.values())

                wickets_player_out = None
                wickets_kind = None
                fielder1_wicket = None
                fielder2_wicket = None
                fielder3_wicket = None

                legbyes_extras = extras.get('legbyes', 0)
                wides_extras = extras.get('wides', 0)
                byes_extras = extras.get('byes', 0)
                noballs_extras = extras.get('noballs', 0)
                penalty_extras = extras.get('penalty', 0)

                if wickets:  # Check if there are wickets in this delivery
                    for wicket in wickets:
                        wickets_player_out = wicket.get('player_out', None)
                        wickets_kind = wicket.get('kind', None)
                        fielders_wicket = wicket.get('fielders', [])
                        fielder1_wicket = None
                        fielder2_wicket = None
                        fielder3_wicket = None

                        # Extract fielder information if available
                        for i, fielder in enumerate(fielders_wicket):
                            if i == 0:
                                fielder1_wicket = fielder.get('name', None)
                            elif i == 1:
                                fielder2_wicket = fielder.get('name', None)
                            elif i == 2:
                                fielder3_wicket = fielder.get('name', None)

                # Extract extra information if available
                extras_keys = list(extras.keys())
                if extras_keys:
                    extras_type1 = extras_keys[0]
                    runs_extras1 = extras.get(extras_type1, None)

                    if len(extras_keys) > 1:
                        extras_type2 = extras_keys[1]
                        runs_extras2 = extras.get(extras_type2, None)

                cursor.execute('''
                    INSERT INTO ball_by_ball_data
                    (match_id, team, innings_number, over_number, powerplay_type, powerplay_number, ball_number,
                    batter, batter_number, non_striker_number, bowler, bowler_number, non_striker, runs_batter, extras_total, target_overs, target_runs, wickets_player_out,
                    wickets_kind, fielder1_wicket, fielder2_wicket, fielder3_wicket, legbyes_extras, wides_extras, byes_extras, noballs_extras, penalty_extras)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''',
                    (match_id, team, count_inn, over_number, pp_type, pp, ball_number, batter_name, batsman_order[batter_name],
                    batsman_order[non_striker_name], bowler_name, bowler_order[bowler_name],
                    non_striker_name, delivery['runs']['batter'], runs_extras,
                    target_overs, target_runs,
                    wickets_player_out, wickets_kind, fielder1_wicket, fielder2_wicket, fielder3_wicket,
                    legbyes_extras, wides_extras, byes_extras, noballs_extras, penalty_extras))
                ball_number += 1

    # Commit the changes and close the database
    conn.commit()

In [ ]:
import glob
files = glob.glob("/content/drive/MyDrive/grayball/all_json/*.json")
print(len(files))

15790


In [ ]:
map_df = []
for file in files:
  with open(file, 'r') as json_file:
      cric_data = json.load(json_file)
  match_id = str(uuid.uuid4())
  insert_match_info(cric_data, match_id)
  map_df.append({"filename": file, "match_id": match_id})

In [ ]:
map_df = pd.DataFrame(map_df)

In [ ]:
map_df.to_csv("/content/drive/MyDrive/grayball/mapping_df.csv", index=False)

In [ ]:
for row in range(len(map_df)):
  with open(map_df.iloc[row]["filename"], 'r') as json_file:
      cric_data = json.load(json_file)
  match_id = map_df.iloc[row]["match_id"]
  insert_ball_by_ball(cric_data, match_id)

In [ ]:
# os.remove("/content/drive/MyDrive/grayball/cricket.db")
# os.path.exists("/content/drive/MyDrive/grayball/cricket.db")

In [ ]:
# conn = sqlite3.connect('/content/drive/MyDrive/grayball/cricket_try.db')
# query = f"""
#       SELECT *
#       FROM ball_by_ball_data b
#       LEFT JOIN match_info m ON b.match_id = m.match_id;
#   """
# batsman_df = pd.read_sql_query(query, conn)

In [ ]:
# import pandas as pd
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# import re

In [ ]:
# display(batsman_df.head(10))